In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
hf_model = AutoModelForCausalLM.from_pretrained(
    "./checkpoints/bigcode/starcoder2-3b", torch_dtype=torch.bfloat16
).to("cuda")

In [5]:
from model import Transformer
from util import load_model
from pathlib import Path

ckpt_path = Path("./checkpoints/bigcode/starcoder2-3b/model.pth")
with torch.device("meta"):
    model = Transformer.from_name(ckpt_path.parent.name)
model = load_model(
    model,
    "./checkpoints/bigcode/starcoder2-3b/model.pth",
    device="cuda",
    precision=torch.bfloat16,
)

In [ ]:
from torch.nn.attention.flex_attention import create_block_mask


def causal_mask(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx


def prefill(
    model: Transformer, x: torch.Tensor, input_pos: torch.Tensor, **sampling_kwargs
) -> torch.Tensor:
    # input_pos: [B, S]
    mask = create_block_mask(
        causal_mask, 1, 1, input_pos.shape[0], model.max_seq_length, device=x.device
    )
    logits = model(mask, x, input_pos)
    return logits


device = torch.device("cuda")
with device:
    with torch.inference_mode():
        input_ids = tokenizer.encode("def foo(x):")
        input_ids = torch.tensor(input_ids).unsqueeze(0)
        input_pos = torch.arange(input_ids.shape[1])
        model.setup_caches(1, 4096)
        logits = prefill(model, input_ids, input_pos)

E0304 05:13:15.236000 3519 torch/_inductor/select_algorithm.py:1477] [0/0] Exception out of resource: shared memory, Required: 164864, Hardware limit: 101376. Reducing block sizes or `num_stages` may help. for benchmark choice TritonTemplateCaller(/tmp/torchinductor_will/so/csoeq4w2c5uyyi4vaacy7fpbij4aql4hbyqwit3xmvqkgdlzio3r.py, BLOCKS_ARE_CONTIGUOUS=False, BLOCK_M=128, BLOCK_N=128, FLOAT32_PRECISION="'ieee'", GQA_SHARED_HEADS=16, HAS_FULL_BLOCKS=True, IS_DIVISIBLE=False, OUTPUT_LOGSUMEXP=False, PRESCALE_QK=False, QK_HEAD_DIM=128, ROWS_GUARANTEED_SAFE=False, SAFE_M_BOUNDARY=False, SAFE_N_BOUNDARY=True, SM_SCALE=0.08838834764831843, SPARSE_KV_BLOCK_SIZE=128, SPLIT_KV=64, V_HEAD_DIM=128, num_stages=3, num_warps=2)
W0304 05:13:15.616000 3519 torch/_inductor/select_algorithm.py:1696] [0/0] out of resource: shared memory, Required: 164864, Hardware limit: 101376. Reducing block sizes or `num_stages` may help.
AUTOTUNE flex_decoding(1x2x16x5x128, 1x2x4096x128, 1x2x4096x128, 1x64x32x5, 1x64x

torch.return_types.topk(
values=tensor([0.0554, 0.0488, 0.0459, 0.0231, 0.0192, 0.0149], device='cuda:0',
       dtype=torch.bfloat16),
indices=tensor([ 913,  640, 2594, 1176,  100,  634], device='cuda:0'))

In [11]:
torch.topk(torch.nn.functional.softmax(logits[0, 0]), 6)

/tmp/ipykernel_3519/241237498.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.topk(torch.nn.functional.softmax(logits[0, 0]), 6)


torch.return_types.topk(
values=tensor([0.0554, 0.0488, 0.0459, 0.0231, 0.0192, 0.0149], device='cuda:0',
       dtype=torch.bfloat16),
indices=tensor([ 913,  640, 2594, 1176,  100,  634], device='cuda:0'))

In [10]:
with torch.inference_mode():
    hf_logits = hf_model(input_ids).logits
torch.topk(torch.nn.functional.softmax(hf_logits[0, 0]), 6)

/tmp/ipykernel_3519/3314666126.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.topk(torch.nn.functional.softmax(hf_logits[0, 0]), 6)


torch.return_types.topk(
values=tensor([0.0562, 0.0466, 0.0439, 0.0234, 0.0208, 0.0151], device='cuda:0',
       dtype=torch.bfloat16),
indices=tensor([ 913,  640, 2594, 1176,  100,  634], device='cuda:0'))

In [ ]:
def causal_mask(b, h, q_idx, kv_idx):
    return q_idx >= kv_idx